In [71]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import pandas as pd

In [72]:
bd.__version__ >= (4, 0, 'DEV7'), bi.__version__ >= (0, 9, 'DEV4'), bc.__version__ >= (2, 0, 'DEV2')

(True, True, True)

In [73]:
bd.projects.set_current("fresh-start")

In [74]:
if "mobility" in bd.databases:
    del bd.databases['mobility']

In [75]:
db = bd.Database("mobility")
db.register()

biosphere = bd.Database("biosphere")
biosphere.register()

In [76]:
data = {
    'code': 'bike',
    'name': 'bike production',
    'location': 'NO',
    'unit': 'bike'
}

bike = db.new_activity(**data)
bike.save()

In [77]:
data = {
    'code': 'CF',
    'name': 'carbon fibre',
    'unit': 'kilogram',
    'location': 'CN'
}

cf = db.new_activity(**data)
cf.save()

In [78]:
ng = db.new_activity(
    name="Nat Gas", 
    code='ng', 
    location='NO', 
    unit='MJ'
)

ng.save()

In [79]:
co2 = biosphere.new_activity(
    name="Carbon Dioxide", 
    code='co2', 
    categories=('air',),
    type='emission',
)

co2.save()

In [80]:
bike.new_exchange(
    amount=2.5, 
    type='technosphere',
    input=cf
).save()

In [81]:
cf.new_exchange(
    amount=237.3,  # plus 58 kWh of electricity, in ecoinvent 3.8 
    uncertainty_type=5, 
    minimum=200, 
    maximum=300, 
    type='technosphere',
    input=ng,
).save()

In [82]:
cf.new_exchange(
    amount=26.6, 
    uncertainty_type=5, 
    minimum=26,
    maximum=27.2, 
    type='biosphere',
    input=co2,
).save()

In [83]:
ipcc = bd.Method(('IPCC',))
ipcc.write([
    (co2.key, {'amount': 1, 'uncertainty_type': 3, 'loc': 1, 'scale': 0.05}),
])

In [84]:
import bw2calc as bc

In [85]:
lca = bc.LCA(
    {bike: 1}, # Func unit is one bike
    ('IPCC',),
    use_distributions=True,
    seed_override=None
)
lca.lci()
lca.lcia()

In [86]:
pd.DataFrame([
    {
        'score': lca.score, 
        'inv': lca.inventory.sum(), 
        # 'heat': lca.technosphere_matrix[lca.dicts.product[ng.id], lca.dicts.activity[cf.id]],
        'char': lca.characterization_matrix.sum()
    } for _, _ in zip(lca, range(10))
])

,score,inv,char
0,62.764451,67.424308,0.930888
1,65.691718,66.581470,0.986637
2,67.156772,66.468136,1.010360
3,64.898436,65.442234,0.991690
4,71.902108,67.474580,1.065618
5,63.825486,67.324096,0.948033
6,67.238281,66.413333,1.012421
7,60.963442,65.378888,0.932464
8,62.009117,67.796617,0.914634
9,73.179534,66.954859,1.092968


What if we only want uncertainty in the characterization step?

We need to modify the datapackage a bit

In [87]:
from bw_processing import load_datapackage
from fs.zipfs import ZipFS

In [88]:
database_dp = load_datapackage(ZipFS(db.filepath_processed()))
method_dp = load_datapackage(ZipFS(ipcc.filepath_processed()))

We can use the datapackages directly, just as before:

In [89]:
lca = bc.LCA(
    {bike.id: 1},
    data_objs=[database_dp, method_dp],
    use_distributions=True,
    seed_override=None
)
lca.lci()
lca.lcia()

In [90]:
pd.DataFrame([
    {
        'score': lca.score, 
        'inv': lca.inventory.sum(), 
        'char': lca.characterization_matrix.sum()
    } for _, _ in zip(lca, range(10))
])

,score,inv,char
0,67.063269,66.643624,1.006297
1,65.682686,66.902617,0.981766
2,69.138147,66.533737,1.039144
3,62.597911,65.435395,0.956637
4,66.807213,67.180669,0.994441
5,63.258171,65.459003,0.966378
6,69.486621,65.331914,1.063594
7,65.188154,66.806162,0.975781
8,72.279078,66.064869,1.094062
9,66.121780,67.396909,0.981080


But we can also filter out the uncertainty from the $A$ matrix:

In [91]:
database_dp.groups

{'mobility_biosphere_matrix': <bw_processing.datapackage.FilteredDatapackage at 0x12cef17c0>,
 'mobility_inventory_geomapping_matrix': <bw_processing.datapackage.FilteredDatapackage at 0x12cef1d60>,
 'mobility_technosphere_matrix': <bw_processing.datapackage.FilteredDatapackage at 0x12cef10d0>}

In [92]:
filtered_dp = (database_dp
               .exclude_resource_group_kind('mobility_technosphere_matrix', 'distributions')
               .exclude_resource_group_kind('mobility_biosphere_matrix', 'distributions')
              )

In [93]:
[obj['name'] for obj in database_dp.resources if obj['kind'] == 'distributions']

['mobility_biosphere_matrix.distributions',
 'mobility_technosphere_matrix.distributions']

In [94]:
[obj['name'] for obj in filtered_dp.resources if obj['kind'] == 'distributions']

[]

In [95]:
lca = bc.LCA(
    {bike.id: 1},
    data_objs=[filtered_dp, method_dp],
    use_distributions=True,
    seed_override=None
)
lca.lci()
lca.lcia()

In [110]:
pd.DataFrame([
    {
        'score': lca.score, 
        'inv': lca.inventory.sum(), 
        'char': lca.characterization_matrix.sum()
    } for _, _ in zip(lca, range(10))
])

,score,inv,char
0,62.673102,66.500001,0.942453
1,69.680132,66.500001,1.047822
2,65.141145,66.500001,0.979566
3,66.346803,66.500001,0.997696
4,66.129796,66.500001,0.994433
5,65.030756,66.500001,0.977906
6,65.826173,66.500001,0.989867
7,64.270620,66.500001,0.966475
8,64.721621,66.500001,0.973257
9,67.362556,66.500001,1.012971


We have a problem in our modelling. Maybe you spotted it already; in any case, we can see this LCI data in a document. This is the third perspective, and so we need to use our third library: `bw2io`.

In [97]:
from bw2io.export import write_lci_excel

In [98]:
from pathlib import Path

In [99]:
xl = Path(write_lci_excel(db.name))

In [100]:
xl.rename(Path.cwd() / xl.name)

PosixPath('/Users/cmutel/Code/brightway/from-the-ground-up/notebooks/lci-mobility.xlsx')

In [101]:
bi.create_core_migrations()

In [102]:
xl_importer = bi.ExcelImporter("lci-mobility.xlsx")

Extracted 1 worksheets in 0.04 seconds


In [103]:
xl_importer.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.02 seconds


In [104]:
xl_importer.statistics()

3 datasets
3 exchanges
1 unlinked exchanges
  Type biosphere: 1 unique unlinked exchanges


(3, 3, 1)

In [105]:
for obj in xl_importer.unlinked:
    print(obj)

{'name': 'Carbon Dioxide', 'amount': 26.6, 'categories': ('air',), 'type': 'biosphere', 'minimum': 26, 'maximum': 27.2, 'uncertainty_type': 5}


In [106]:
co2

'Carbon Dioxide' (None, GLO, ('air',))

In [107]:
xl_importer.match_database("biosphere", fields=['name'])

Applying strategy: link_iterable_by_fields


In [108]:
xl_importer.statistics()

3 datasets
3 exchanges
0 unlinked exchanges
  


(3, 3, 0)

In [109]:
xl_importer.write_database()

Writing activities to SQLite3 database:
0% [###] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/18/2021 23:01:41
  Finished: 10/18/2021 23:01:41
  Total time elapsed: 00:00:00
  CPU %: 52.50
  Memory %: 1.80
Created database: mobility
